In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.feature_selection import SelectKBest, chi2, f_classif

# Đọc file đã làm sạch
df = pd.read_csv('Cleaned_Data.csv')

# Định nghĩa đặc trưng số và phân loại
numeric_cols = ['Income', 'Age', 'Experience', 'CURRENT_JOB_YRS', 'CURRENT_HOUSE_YRS']
categorical_features = ['Profession', 'City', 'STATE', 'Married/Single', 'Car_Ownership', 'House_Ownership']

# 1. Đặc trưng số (numeric): chuẩn hóa
scaler = StandardScaler()
df[numeric_cols] = scaler.fit_transform(df[numeric_cols])

# 2. Đặc trưng nhị phân (2 giá trị): Label Encoding cho Married/Single và Car_Ownership
df['Married/Single'] = df['Married/Single'].str.lower().map({'single': 0, 'married': 1})
df['Car_Ownership'] = df['Car_Ownership'].map({'no': 0, 'yes': 1})

# 3. One-Hot Encoding cho House_Ownership
house_dummies = pd.get_dummies(df['House_Ownership'], prefix='House_Ownership').astype(int)
df = pd.concat([df, house_dummies], axis=1)
df.drop('House_Ownership', axis=1, inplace=True)

# 4. Chuẩn hóa Profession, City, STATE
def frequency_encoding(df, column):
    freq = df[column].value_counts(normalize=True)
    return df[column].map(freq)

for col in ['Profession', 'City', 'STATE']:
    df[col] = frequency_encoding(df, col)

# 5. Label Encoding cho cột Risk_Flag
le_risk_flag = LabelEncoder()
df['Risk_Flag'] = df['Risk_Flag'].str.lower().map({'safe': 0, 'risky': 1})

# In ra kết quả sau khi xử lý đặc trưng
print("Dữ liệu sau khi xử lý đặc trưng:")
print(df.head())


Dữ liệu sau khi xử lý đặc trưng:
     Income       Age  Experience  Married/Single  Car_Ownership  Profession  \
0  0.907864 -0.584421    0.123178               0              0    0.020052   
1 -0.356528  0.940960   -0.050777               1              0    0.020616   
2  0.442698 -0.525753   -1.268468               0              1    0.020052   
3  0.058096 -0.173742    0.297134               0              0    0.017520   
4  0.675441  0.823623   -0.050777               0              0    0.017520   

   CURRENT_JOB_YRS  CURRENT_HOUSE_YRS  Risk_Flag      City     STATE  \
0         0.819348           0.716325          0  0.003370  0.101456   
1        -0.593559          -1.428003          0  0.002731  0.023041   
2        -1.158722           0.001549          1  0.002409  0.018477   
3        -0.876140           1.431102          1  0.003211  0.065630   
4        -1.723885           0.001549          0  0.003402  0.101456   

   House_Ownership_norent_noown  House_Ownership_owne

In [ ]:
# 6. Phương pháp 1: Chi-Square Test cho các đặc trưng phân loại
categorical_features = ['Married/Single', 'Car_Ownership', 'House_Ownership_norent_noown',
                       'House_Ownership_owned', 'House_Ownership_rented']
X_chi2 = df[categorical_features].copy()
X_chi2 = X_chi2.apply(lambda x: x + abs(x.min()) if x.min() < 0 else x)

chi2_selector = SelectKBest(score_func=chi2, k='all')
chi2_selector.fit(X_chi2, df['Risk_Flag'])
chi2_scores = pd.DataFrame({
    'Feature': categorical_features,
    'Chi2 Score': chi2_selector.scores_,
    'P-value': chi2_selector.pvalues_
})
significant_chi2_features = chi2_scores[chi2_scores['P-value'] <= 0.05]['Feature'].tolist()

# In đặc trưng được chọn từ Chi-Square Test
print("\nĐặc trưng được chọn từ Chi-Square Test (p-value <= 0.05):")
print(significant_chi2_features)
print("\nChi tiết điểm số Chi-Square Test:")
print(chi2_scores)


Đặc trưng được chọn từ Chi-Square Test (p-value <= 0.05):
['Married/Single', 'Car_Ownership', 'House_Ownership_norent_noown', 'House_Ownership_owned', 'House_Ownership_rented']

Chi tiết điểm số Chi-Square Test:
                        Feature  Chi2 Score       P-value
0                Married/Single  101.410712  7.475843e-24
1                 Car_Ownership   74.949690  4.828630e-18
2  House_Ownership_norent_noown   32.323878  1.304998e-08
3         House_Ownership_owned  116.222131  4.249335e-27
4        House_Ownership_rented   12.288264  4.558154e-04


In [ ]:
# 7. Phương pháp 2: ANOVA (F-test) cho đặc trưng số
numeric_cols = ['Income', 'Age', 'Experience', 'CURRENT_JOB_YRS', 'CURRENT_HOUSE_YRS',
                'Profession', 'City', 'STATE']
anova_selector = SelectKBest(score_func=f_classif, k='all')
anova_selector.fit(df[numeric_cols], df['Risk_Flag'])
anova_scores = pd.DataFrame({
    'Feature': numeric_cols,
    'F-Score': anova_selector.scores_,
    'P-value': anova_selector.pvalues_
})
significant_anova_features = anova_scores[anova_scores['P-value'] <= 0.05]['Feature'].tolist()

# In đặc trưng được chọn từ ANOVA
print("\nĐặc trưng được chọn từ ANOVA (F-test) (p-value <= 0.05):")
print(significant_anova_features)
print("\nChi tiết điểm số ANOVA (F-test):")
print(anova_scores)


Đặc trưng được chọn từ ANOVA (F-test) (p-value <= 0.05):
['Age', 'Experience', 'CURRENT_JOB_YRS', 'CURRENT_HOUSE_YRS', 'Profession', 'City']

Chi tiết điểm số ANOVA (F-test):
             Feature     F-Score        P-value
0             Income    2.529796   1.117159e-01
1                Age  118.689450   1.242319e-27
2         Experience  458.254284  1.420934e-101
3    CURRENT_JOB_YRS   98.234505   3.753050e-23
4  CURRENT_HOUSE_YRS    4.924502   2.647932e-02
5         Profession   18.778443   1.468740e-05
6               City   11.962709   5.428486e-04
7              STATE    0.012485   9.110320e-01


In [ ]:
# 8. Tổng hợp các đặc trưng quan trọng từ cả hai phương pháp
selected_features = list(set(significant_chi2_features + significant_anova_features))
print("\nCác đặc trưng cuối cùng được chọn:")
print(selected_features)


Các đặc trưng cuối cùng được chọn:
['Married/Single', 'CURRENT_JOB_YRS', 'City', 'Experience', 'House_Ownership_owned', 'Profession', 'CURRENT_HOUSE_YRS', 'House_Ownership_norent_noown', 'Car_Ownership', 'House_Ownership_rented', 'Age']


In [ ]:
# 9. Gán X và y
X = df[selected_features]
y = df['Risk_Flag']

# In kết quả tổng hợp
print("\nĐặc trưng (X) sau khi chọn lọc:")
print(X.head())
print("\nNhãn (y):")
print(y.head())



Đặc trưng (X) sau khi chọn lọc:
   Married/Single  CURRENT_JOB_YRS      City  Experience  \
0               0         0.819348  0.003370    0.123178   
1               1        -0.593559  0.002731   -0.050777   
2               0        -1.158722  0.002409   -1.268468   
3               0        -0.876140  0.003211    0.297134   
4               0        -1.723885  0.003402   -0.050777   

   House_Ownership_owned  Profession  CURRENT_HOUSE_YRS  \
0                      0    0.020052           0.716325   
1                      0    0.020616          -1.428003   
2                      0    0.020052           0.001549   
3                      0    0.017520           1.431102   
4                      0    0.017520           0.001549   

   House_Ownership_norent_noown  Car_Ownership  House_Ownership_rented  \
0                             0              0                       1   
1                             0              0                       1   
2                            

In [ ]:
from imblearn.combine import SMOTETomek

# Cân bằng dữ liệu toàn bộ trước khi chia
smote_tomek = SMOTETomek(random_state=42)
X_resampled, y_resampled = smote_tomek.fit_resample(X, y)

# Kiểm tra phân phối lớp
print("Phân phối lớp sau SMOTE + Tomek Links:")
print(pd.Series(y_resampled).value_counts(normalize=True))


Phân phối lớp sau SMOTE + Tomek Links:
Risk_Flag
0    0.5
1    0.5
Name: proportion, dtype: float64


In [ ]:
# Chia tập train và test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, stratify=y_resampled, random_state=42) # Use y_resampled for stratification


In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
import time

# Đo thời gian bắt đầu
start_time = time.time()

# Khởi tạo và huấn luyện mô hình Decision Tree
model = DecisionTreeClassifier(
    random_state=42,        # để kết quả có thể lặp lại
    class_weight='balanced' # (tuỳ chọn) nếu dữ liệu imbalanced
)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)


print("\nDecision Tree Report:")
print(classification_report(y_test, y_pred))

# Đo thời gian thực thi
DT_ex_time = time.time() - start_time
print(f"Thời gian thực thi: {DT_ex_time:.2f} giây")


Decision Tree Report:
              precision    recall  f1-score   support

           0       0.94      0.90      0.92     43551
           1       0.90      0.94      0.92     43551

    accuracy                           0.92     87102
   macro avg       0.92      0.92      0.92     87102
weighted avg       0.92      0.92      0.92     87102

Thời gian thực thi: 6.59 giây


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import time  # nhớ import time nếu chưa import

# Đo thời gian bắt đầu
RS_stime = time.time()

# Tạo và huấn luyện mô hình
model = RandomForestClassifier(class_weight='balanced', random_state=42)
model.fit(X_train, y_train)


y_pred_brf = model.predict(X_test)

print("Balanced Random Forest Report:")
print(classification_report(y_test, y_pred_brf))

# Đo thời gian thực thi
RF_ex_time = time.time() - RS_stime
print(f"Thời gian thực thi: {RF_ex_time:.2f} giây")


Balanced Random Forest Report:
              precision    recall  f1-score   support

           0       0.96      0.92      0.94     43551
           1       0.92      0.96      0.94     43551

    accuracy                           0.94     87102
   macro avg       0.94      0.94      0.94     87102
weighted avg       0.94      0.94      0.94     87102

Thời gian thực thi: 82.10 giây
